# 🔄 Map, FlatMap & Filter: Core RDD Transformations

**Time to complete:** 30 minutes  
**Difficulty:** Intermediate  
**Prerequisites:** RDD basics, transformations vs actions

---

## 🎯 Learning Objectives

By the end of this notebook, you will master:
- ✅ **map()** - Transform each element one-to-one
- ✅ **flatMap()** - Transform and flatten results
- ✅ **filter()** - Select elements based on conditions
- ✅ **flatMap vs map** - When to use each
- ✅ **Performance implications** - Choosing the right transformation
- ✅ **Real-world use cases** - Practical applications

**These are the most fundamental RDD operations you'll use daily!**

---

## 🔍 Understanding the Core Transformations

`map()`, `flatMap()`, and `filter()` are the **bread and butter** of RDD operations. They form the foundation for most data processing pipelines.

### Quick Comparison:

| Operation | Input:Output | Use Case |
|-----------|-------------|----------|
| `map()` | 1 → 1 | Transform each element |
| `flatMap()` | 1 → Many | Split/combine elements |
| `filter()` | 1 → 0 or 1 | Select elements |

**All are narrow transformations** - no data shuffling required! ⚡

## ⚙️ Setup

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MapFlatMapFilter") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

print(f"✅ Spark ready - Version: {spark.version}")
print(f"✅ Using {sc.defaultParallelism} partitions")

## 🗺️ Map Transformation

**`map(func)` applies a function to each element and returns a new RDD with the results.**

### Characteristics:
- **One-to-one transformation**: Each input produces exactly one output
- **Same number of elements**: Input count = Output count
- **Type transformation**: Can change element types
- **Lazy evaluation**: Doesn't execute until action called

In [ ]:
# Create sample data
numbers = [1, 2, 3, 4, 5]
numbers_rdd = sc.parallelize(numbers)

print("📊 MAP TRANSFORMATION EXAMPLES")
print("=" * 50)
print(f"Original RDD: {numbers_rdd.collect()}")

# Example 1: Double each number
doubled = numbers_rdd.map(lambda x: x * 2)
print(f"\nDoubled (x * 2): {doubled.collect()}")

# Example 2: Square each number
squared = numbers_rdd.map(lambda x: x ** 2)
print(f"Squared (x²): {squared.collect()}")

# Example 3: Convert to string with formatting
formatted = numbers_rdd.map(lambda x: f"Number: {x}")
print(f"Formatted strings: {formatted.collect()}")

# Example 4: Type conversion
float_numbers = numbers_rdd.map(lambda x: float(x))
print(f"Converted to float: {float_numbers.collect()}")
print(f"Type changed: {type(numbers_rdd.collect()[0])} → {type(float_numbers.collect()[0])}")

### Advanced Map Examples

In [ ]:
# Working with complex data
employees = [
    ("Alice", 25, 50000),
    ("Bob", 30, 60000),
    ("Charlie", 35, 70000)
]

employees_rdd = sc.parallelize(employees)
print(f"\nEmployee data: {employees_rdd.collect()}")

# Extract just names
names_only = employees_rdd.map(lambda emp: emp[0])
print(f"Names only: {names_only.collect()}")

# Calculate bonus (10% of salary)
with_bonus = employees_rdd.map(lambda emp: (emp[0], emp[2], emp[2] * 0.1))
print(f"With bonus: {with_bonus.collect()}")

# Create employee summary
summaries = employees_rdd.map(
    lambda emp: f"{emp[0]} is {emp[1]} years old and earns ${emp[2]:,}"
)
print(f"\nEmployee summaries:")
for summary in summaries.collect():
    print(f"  {summary}")

## 🗂️ FlatMap Transformation

**`flatMap(func)` applies a function to each element and flattens the results.**

### Characteristics:
- **One-to-many transformation**: Each input can produce multiple outputs
- **Flattening**: Nested structures become flat
- **Variable output count**: Input count ≤ Output count
- **Perfect for splitting**: Text processing, expanding collections

In [ ]:
# Text processing examples
sentences = [
    "Hello world welcome to Spark",
    "PySpark is powerful and fast",
    "RDD transformations are fun"
]

sentences_rdd = sc.parallelize(sentences)

print("📖 FLATMAP TRANSFORMATION EXAMPLES")
print("=" * 50)
print("Original sentences:")
for i, sentence in enumerate(sentences_rdd.collect(), 1):
    print(f"  {i}. '{sentence}'")

# Example 1: Split sentences into words
words = sentences_rdd.flatMap(lambda sentence: sentence.split())
print(f"\nAfter flatMap(split): {words.collect()}")
print(f"Word count: {words.count()} (was {sentences_rdd.count()} sentences)")

# Example 2: Multiple words per input
multi_words = sentences_rdd.flatMap(lambda s: [s.upper(), s.lower()])
print(f"\nMultiple outputs per input: {multi_words.collect()}")
print(f"Output count: {multi_words.count()} (2 × input count)")

### FlatMap vs Map Comparison

In [ ]:
print("🔍 FLATMAP vs MAP COMPARISON")
print("=" * 50)

# Same input
test_sentences = sc.parallelize(["Hello world", "Spark is great"])

# Using map (splits but keeps nested structure)
map_result = test_sentences.map(lambda s: s.split())
print(f"map(split): {map_result.collect()}")
print(f"Type: {type(map_result.collect()[0])} - Nested lists")

# Using flatMap (splits and flattens)
flatmap_result = test_sentences.flatMap(lambda s: s.split())
print(f"\nflatMap(split): {flatmap_result.collect()}")
print(f"Type: {type(flatmap_result.collect()[0])} - Flat list")

# Word count example
text = sc.parallelize(["Big data processing", "Spark RDD operations"])
word_count_map = text.map(lambda line: len(line.split()))
word_count_flatmap = text.flatMap(lambda line: line.split()).count()

print(f"\nmap() result: {word_count_map.collect()} (words per line)")
print(f"flatMap() result: {word_count_flatmap} (total words)")

### Advanced FlatMap Use Cases

In [ ]:
# Expanding collections
nested_data = [
    ["apple", "banana", "cherry"],
    ["dog", "cat"],
    ["red", "green", "blue", "yellow"]
]

nested_rdd = sc.parallelize(nested_data)
print(f"\nNested data: {nested_rdd.collect()}")

# Flatten the nested lists
flattened = nested_rdd.flatMap(lambda x: x)
print(f"Flattened: {flattened.collect()}")

# Generate multiple related items
products = sc.parallelize(["laptop", "phone"])
product_features = products.flatMap(lambda p: [f"{p}_price", f"{p}_rating", f"{p}_reviews"])
print(f"\nProduct features: {product_features.collect()}")

# Text analysis - n-grams
sentence = sc.parallelize(["The quick brown fox jumps"])
bigrams = sentence.flatMap(lambda s: [f"{s.split()[i]} {s.split()[i+1]}" 
                                       for i in range(len(s.split())-1)])
print(f"\nBigrams: {bigrams.collect()}")

## 🔍 Filter Transformation

**`filter(func)` keeps only elements that satisfy a condition.**

### Characteristics:
- **Conditional selection**: Elements pass through or are dropped
- **Zero-to-one transformation**: Each input produces 0 or 1 output
- **Reduced dataset**: Output count ≤ Input count
- **Predicate function**: Returns True (keep) or False (drop)

In [ ]:
# Numeric filtering
all_numbers = sc.parallelize(range(1, 21))  # 1 to 20

print("🔍 FILTER TRANSFORMATION EXAMPLES")
print("=" * 50)
print(f"All numbers: {all_numbers.collect()}")

# Example 1: Even numbers only
evens = all_numbers.filter(lambda x: x % 2 == 0)
print(f"\nEven numbers: {evens.collect()}")

# Example 2: Numbers greater than 10
big_numbers = all_numbers.filter(lambda x: x > 10)
print(f"Numbers > 10: {big_numbers.collect()}")

# Example 3: Prime numbers (basic check)
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = all_numbers.filter(is_prime)
print(f"Prime numbers: {primes.collect()}")

# Example 4: Complex conditions
special = all_numbers.filter(lambda x: x % 3 == 0 and x > 5)
print(f"Multiples of 3 > 5: {special.collect()}")

### String and Complex Filtering

In [ ]:
# Text filtering
words_list = [
    "hello", "world", "spark", "data", "python", "big",
    "small", "fast", "slow", "machine", "learning", "AI"
]

words_rdd = sc.parallelize(words_list)
print(f"\nAll words: {words_rdd.collect()}")

# Filter by length
long_words = words_rdd.filter(lambda w: len(w) > 5)
print(f"Words > 5 chars: {long_words.collect()}")

# Filter by starting letter
starts_with_s = words_rdd.filter(lambda w: w.startswith('s'))
print(f"Words starting with 's': {starts_with_s.collect()}")

# Complex object filtering
products = [
    {"name": "Laptop", "price": 1200, "category": "Electronics"},
    {"name": "Book", "price": 25, "category": "Education"},
    {"name": "Chair", "price": 150, "category": "Furniture"},
    {"name": "Phone", "price": 800, "category": "Electronics"}
]

products_rdd = sc.parallelize(products)

# Filter expensive electronics
expensive_electronics = products_rdd.filter(
    lambda p: p["category"] == "Electronics" and p["price"] > 500
)

print(f"\nExpensive electronics:")
for product in expensive_electronics.collect():
    print(f"  {product['name']}: ${product['price']}")

## 🔗 Chaining Transformations

**Combine map, flatMap, and filter for powerful data processing pipelines.**

In [ ]:
# Text processing pipeline
raw_text = [
    "Hello World! Welcome to PySpark.",
    "This is a SAMPLE text with Numbers 123.",
    "Spark RDDs are powerful tools!"
]

text_rdd = sc.parallelize(raw_text)

print("🔗 TRANSFORMATION CHAINING EXAMPLES")
print("=" * 50)
print("Raw text:")
for i, line in enumerate(text_rdd.collect(), 1):
    print(f"  {i}. '{line}'")

# Complete text processing pipeline
processed_words = text_rdd \
    .flatMap(lambda line: line.split()) \
    .map(lambda word: word.lower().strip('.,!')) \
    .filter(lambda word: len(word) > 2) \
    .filter(lambda word: word.isalpha()) \
    .distinct()

print(f"\nProcessed unique words: {sorted(processed_words.collect())}")

# Word frequency analysis
word_freq = text_rdd \
    .flatMap(lambda line: line.split()) \
    .map(lambda word: word.lower().strip('.,!')) \
    .filter(lambda word: word.isalpha()) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .filter(lambda pair: pair[1] > 1)  # Only words that appear multiple times

print(f"\nWords appearing multiple times:")
for word, count in sorted(word_freq.collect()):
    print(f"  {word}: {count}")

## ⚡ Performance Comparison

**Understanding the performance characteristics of each transformation.**

In [ ]:
import time

# Create large dataset for performance testing
large_data = list(range(1, 100001))  # 100,000 numbers
large_rdd = sc.parallelize(large_data)

print("⚡ PERFORMANCE COMPARISON")
print("=" * 50)
print(f"Dataset size: {large_rdd.count():,} elements")

# Test map performance
start = time.time()
map_result = large_rdd.map(lambda x: x * 2).count()
map_time = time.time() - start

# Test filter performance
start = time.time()
filter_result = large_rdd.filter(lambda x: x % 1000 == 0).count()
filter_time = time.time() - start

# Test flatMap performance
start = time.time()
flatmap_result = large_rdd.flatMap(lambda x: [x, x*2]).count()
flatmap_time = time.time() - start

print(f"\nmap() time: {map_time:.3f}s (result: {map_result:,})")
print(f"filter() time: {filter_time:.3f}s (result: {filter_result})")
print(f"flatMap() time: {flatmap_time:.3f}s (result: {flatmap_result:,})")

print("\n💡 All are narrow transformations - no data shuffling!")
print("💡 Performance depends on function complexity, not operation type.")

## 🎯 Best Practices & Common Patterns

### When to Use Each Transformation:

In [ ]:
# Pattern examples

# 1. MAP: One-to-one transformations
data = sc.parallelize([1, 2, 3, 4, 5])
squares = data.map(lambda x: x ** 2)  # Each input → one output

# 2. FLATMAP: Splitting or expanding
sentences = sc.parallelize(["hello world", "spark rules"])
words = sentences.flatMap(lambda s: s.split())  # One input → multiple outputs

# 3. FILTER: Selection based on conditions
numbers = sc.parallelize(range(1, 21))
evens = numbers.filter(lambda x: x % 2 == 0)  # Keep only even numbers

# 4. Common pattern: Word count
text = sc.parallelize(["big data spark", "spark is fast"])
word_count = text \
    .flatMap(lambda line: line.split()) \
    .filter(lambda word: len(word) > 0) \
    .map(lambda word: (word.lower(), 1)) \
    .reduceByKey(lambda a, b: a + b)

print("🎯 COMMON PATTERNS")
print("=" * 50)
print(f"Squares: {squares.collect()}")
print(f"Words: {words.collect()}")
print(f"Evens: {evens.collect()}")
print(f"Word count: {dict(word_count.collect())}")

## 🚨 Common Mistakes & Debugging

In [ ]:
# Mistake examples and fixes

print("🚨 COMMON MISTAKES")
print("=" * 50)

# Mistake 1: Using map when you need flatMap
wrong = sc.parallelize(["hello world"]).map(lambda s: s.split())
print(f"Wrong - map(split): {wrong.collect()}  # Nested list")

correct = sc.parallelize(["hello world"]).flatMap(lambda s: s.split())
print(f"Correct - flatMap(split): {correct.collect()}  # Flat list")

# Mistake 2: Forgetting to handle edge cases in filter
data_with_nulls = sc.parallelize(["hello", None, "world", ""])
# wrong_filter = data_with_nulls.filter(lambda x: len(x) > 0)  # Error!

safe_filter = data_with_nulls.filter(lambda x: x is not None and len(x) > 0)
print(f"\nSafe filter result: {safe_filter.collect()}")

# Mistake 3: Complex operations in transformations
numbers = sc.parallelize(range(1, 11))
# Avoid this in production:
complex_map = numbers.map(lambda x: sum(range(x)))  # Very slow!
print(f"\nComplex operation result: {complex_map.collect()}")
print("💡 Move complex logic outside transformations when possible!")

## 🧹 Cleanup

In [ ]:
# Clean up sample files if they exist
import os
sample_files = ["sample_text.txt", "sample_data.csv"]
for file in sample_files:
    if os.path.exists(file):
        os.remove(file)

print("🧹 Cleanup completed")

# Note: Spark Session will be cleaned up automatically in Jupyter
# In production code, use: spark.stop()

## 🎯 Key Takeaways

### What You Learned:
- ✅ **`map()`** - Transform each element one-to-one
- ✅ **`flatMap()`** - Transform and flatten results (one-to-many)
- ✅ **`filter()`** - Select elements based on conditions
- ✅ **Narrow transformations** - No data shuffling required
- ✅ **Chaining** - Combine for powerful data processing pipelines

### Performance Characteristics:
- 🔸 **All are fast** - No network communication
- 🔸 **Parallel execution** - Operations run on multiple cores
- 🔸 **Memory efficient** - Process elements as streams
- 🔸 **Composable** - Build complex pipelines easily

### Common Use Cases:
- 🔸 **map()**: Data type conversion, simple transformations
- 🔸 **flatMap()**: Text splitting, expanding collections
- 🔸 **filter()**: Data cleaning, conditional selection

### Best Practices:
- 🔸 Use `flatMap()` when splitting data (text processing)
- 🔸 Combine transformations to minimize passes through data
- 🔸 Handle edge cases in filter conditions
- 🔸 Keep transformation functions simple and testable

---

## 🚀 Next Steps

Now that you master basic transformations, you're ready for:

1. **Key-Value Operations** - `groupByKey` vs `reduceByKey`
2. **Advanced Aggregations** - `combineByKey` patterns
3. **Partitioning** - `repartition` vs `coalesce`
4. **Wide Transformations** - Understanding shuffle operations

**These form the foundation of 90% of your RDD operations!**

---

**🎉 Congratulations! You now command the core RDD transformations that power distributed data processing!**